# Deployment and Execution of an Electronic Structure Simulation Using SQD IEF-PCM on Qiskit Serverless


In [6]:
# Establish Qiskit Runtime connection
from qiskit_ibm_catalog import QiskitFunction, QiskitServerless

channel  = 'ibm_cloud'
instance = '' #removed for security
token    = '' #removed for security


serverless = QiskitServerless(channel  = channel ,
                          instance = instance,
                          token    = token )

### Template Upload

For this project, the SQD IEF-PCM workflow was uploaded to Qiskit Serverless as a custom Qiskit Function. A `QiskitFunction` object was instantiated to define the function source code, with the title used to identify the function on the remote cluster. The main entry point was the file containing `if __name__ == "__main__"`, and any additional source files required by the workflow were included via a defined working directory 'source_files'.


In [16]:
from qiskit_ibm_catalog import QiskitFunction

template = QiskitFunction(
    title="sqd_pcm_template",
    entrypoint="sqd_pcm_entrypoint.py",
    working_dir="./source_files/",  # all files in this directory will be uploaded
    dependencies=[
        "ffsim==0.0.60",
        "pyscf==2.11.0"
        #"qiskit_addon_sqd==0.10.0",
    ],
)
print(template)

QiskitFunction(sqd_pcm_template)


#### Once the Qiskit Function instance was prepared, it was uploaded to the Serverless environment for execution.

In [17]:
serverless.upload(template)

QiskitFunction(sqd_pcm_template)

#### The successful upload of the Qiskit Function was verified by listing the functions in the Serverless environment using `serverless.list()`.

In [18]:
serverless.list()

[QiskitFunction(classical_simulation), QiskitFunction(sqd_pcm_template)]

## Remote Loading and Execution of the Template

After uploading, the SQD IEF-PCM function template was loaded by name on Qiskit Serverless and executed remotely.


In [19]:
template = serverless.load("sqd_pcm_template")
print(template)

QiskitFunction(sqd_pcm_template)


#### Next The SQD IEF-PCM template was executed using domain-level inputs corresponding to a methanol-based workload.


In [23]:
molecule = {
    "atom": """
    O -0.04559 -0.75076 -0.00000;
    C -0.04844 0.65398 -0.00000;
    H 0.85330 -1.05128 -0.00000;
    H -1.08779 0.98076 -0.00000;
    H 0.44171 1.06337 0.88811;
    H 0.44171 1.06337 -0.88811
    """,  # Must be specified
    "basis": "cc-pvdz",  # default is "sto-3g"
    "spin": 0,  # default is 0
    "charge": 0,  # default is 0
    "verbosity": 0,  # default is 0
    "number_of_active_orb": 12,  # Must be specified
    "number_of_active_alpha_elec": 7,  # Must be specified
    "number_of_active_beta_elec": 7,  # Must be specified
    "avas_selection": [
        "%d O %s" % (k, x) for k in [0] for x in ["2s", "2px", "2py", "2pz"]
    ]
    + ["%d C %s" % (k, x) for k in [1] for x in ["2s", "2px", "2py", "2pz"]]
    + ["%d H 1s" % k for k in [2, 3, 4, 5]],  # default is None
}

solvent_options = {
    "method": "IEF-PCM",  # other available methods are COSMO, C-PCM, SS(V)PE, see https://manual.q-chem.com/5.4/topic_pcm-em.html
    "eps": 78.3553,  # value for water
}

lucj_options = {
    "initial_layout": [
        0,
        14,
        18,
        19,
        20,
        33,
        39,
        40,
        41,
        53,
        60,
        61,
        2,
        3,
        4,
        15,
        22,
        23,
        24,
        34,
        43,
        44,
        45,
        54,
    ],
    "dynamical_decoupling_choice": True,
    "twirling_choice": True,
    "number_of_shots": 200000,
    "optimization_level": 2,
}

sqd_options = {
    "sqd_iterations": 3,
    "number_of_batches": 10,
    "samples_per_batch": 1000,
    "max_davidson_cycles": 200,
}

backend_name = "ibm_torino"

In [24]:
job = template.run(
    backend_name=backend_name,
    molecule=molecule,
    solvent_options=solvent_options,
    lucj_options=lucj_options,
    sqd_options=sqd_options,
)
print(job.job_id)

f5dae434-e589-4b11-918a-b36ce4ed7243


#### The detailed status of the job was monitored to track progress and verify successful execution.


In [25]:
import time

t0 = time.time()
status = job.status()
if status == "QUEUED":
    print(f"time = {time.time()-t0:.2f}, status = QUEUED")
while True:
    status = job.status()
    if status == "QUEUED":
        continue
    print(f"time = {time.time()-t0:.2f}, status = {status}")
    if status == "DONE" or status == "ERROR":
        break

time = 1.34, status = QUEUED
time = 36.26, status = INITIALIZING
time = 37.80, status = INITIALIZING
time = 38.92, status = INITIALIZING
time = 40.25, status = RUNNING: MAPPING
time = 41.48, status = RUNNING: MAPPING
time = 42.82, status = RUNNING: MAPPING
time = 44.18, status = RUNNING: MAPPING
time = 45.47, status = RUNNING: MAPPING
time = 46.70, status = RUNNING: OPTIMIZING_FOR_HARDWARE
time = 47.93, status = RUNNING: OPTIMIZING_FOR_HARDWARE
time = 49.78, status = RUNNING: EXECUTING_QPU
time = 51.11, status = RUNNING: EXECUTING_QPU
time = 52.71, status = RUNNING: EXECUTING_QPU
time = 53.74, status = RUNNING: EXECUTING_QPU
time = 54.90, status = RUNNING: EXECUTING_QPU
time = 56.27, status = RUNNING: EXECUTING_QPU
time = 57.47, status = RUNNING: EXECUTING_QPU
time = 58.79, status = RUNNING: EXECUTING_QPU
time = 60.05, status = RUNNING: EXECUTING_QPU
time = 61.55, status = RUNNING: EXECUTING_QPU
time = 62.84, status = RUNNING: EXECUTING_QPU
time = 64.42, status = RUNNING: EXECUTING_QPU

During execution, logs generated from `logger.info` were collected to monitor the progress of the SQD IEF-PCM workflow. These logs provided insights into details such as same-spin orbital connections and the two-qubit depth of the final ISA circuit prepared for hardware execution.


In [26]:
print(job.logs())

2025-12-12 15:42:00,785	INFO job_manager.py:568 -- Runtime env is setting up.
Running entrypoint for job raysubmit_sdwFzwpgfisdqSuy: python sqd_pcm_entrypoint.py
sqd_pcm_entrypoint.run_function:INFO:2025-12-12 15:42:05,020: Starting runtime service
sqd_pcm_entrypoint.run_function:INFO:2025-12-12 15:42:05,950: Backend: ibm_torino
sqd_pcm_entrypoint.run_function:INFO:2025-12-12 15:42:07,186: Initializing molecule object
sqd_pcm_entrypoint.run_function:INFO:2025-12-12 15:42:11,825: Performing CCSD
Parsing /tmp/ray/session_2025-12-12_15-41-34_339130_1/runtime_resources/working_dir_files/_ray_pkg_9d6482a3112892fc/output_sqd_pcm/2025-12-12_15-42-05.fcidump.txt
Overwritten attributes  get_hcore get_ovlp  of <class 'pyscf.scf.hf_symm.SymAdaptedRHF'>
/usr/local/lib/python3.11/site-packages/pyscf/gto/mole.py:1300: UserWarning: Function mol.dumps drops attribute energy_nuc because it is not JSON-serializable
  warnings.warn(msg)
/usr/local/lib/python3.11/site-packages/pyscf/gto/mole.py:1300: User

Calling for the job result blocks the rest of the program until a result is available. After the job is done, you can retrieve the results. These include the solvation free energy, as well as information about the lowest energy batch, lowest energy value, and other useful information such as the total solver duration.



Note that the result metadata includes a resource usage summary that lets you better estimate the QPU and CPU time required for each workload (this example ran on a dummy device, so actual resource usage times might differ).



In [17]:
print(job.logs())

2025-06-27 08:42:41,358	INFO job_manager.py:531 -- Runtime env is setting up.
sqd_pcm_entrypoint.run_function:INFO:2025-06-27 08:42:45,015: Starting runtime service
sqd_pcm_entrypoint.run_function:INFO:2025-06-27 08:42:45,621: Backend: ibm_sherbrooke
sqd_pcm_entrypoint.run_function:INFO:2025-06-27 08:42:46,809: Initializing molecule object
sqd_pcm_entrypoint.run_function:INFO:2025-06-27 08:42:51,599: Performing CCSD
Parsing /tmp/ray/session_2025-06-27_08-42-13_898146_1/runtime_resources/working_dir_files/_ray_pkg_4bc93dcc58c04b91/output_sqd_pcm/2025-06-27_08-42-45.fcidump.txt
Overwritten attributes  get_ovlp get_hcore  of <class 'pyscf.scf.hf_symm.SymAdaptedRHF'>
/usr/local/lib/python3.11/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute energy_nuc because it is not JSON-serializable
  warnings.warn(msg)
/usr/local/lib/python3.11/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute intor_symmetric because it is not JS

In [27]:
result = job.result()

result

{'total_energy_hist': array([[-115.13667143, -115.13759184, -115.14351858, -115.14571492,
         -115.14997496, -115.14163673, -115.14583982, -115.15535747,
         -115.1264137 , -115.15646478],
        [-115.15832439, -115.1583404 , -115.15837284, -115.15731135,
         -115.15846721, -115.15794118, -115.15830182, -115.15852868,
         -115.15853488, -115.15840969],
        [-115.15860275, -115.15827499, -115.15848818, -115.15824337,
         -115.1585127 , -115.15827924, -115.15816696, -115.15845436,
         -115.15851042, -115.15813318]]),
 'spin_squared_value_hist': array([[1.05327747e-02, 8.89479525e-03, 8.23194158e-03, 7.85440449e-03,
         5.13777403e-03, 1.05646381e-02, 5.42623752e-03, 1.67268213e-03,
         1.45495598e-02, 1.40808896e-03],
        [2.70293354e-04, 2.55529746e-04, 2.08656512e-04, 8.46577712e-04,
         1.65464735e-04, 4.42767478e-04, 2.62080337e-04, 1.08326259e-04,
         1.34130969e-04, 1.77359426e-04],
        [8.66503392e-05, 2.59705852e-04,

After the job completes, the entire logging output will be available.



## SQD IEF-PCM Simulation Results for Methanol in Implicit Solvent

**Energy and Solvation**
- **Lowest-energy batch:** 0
- **Total SQD IEF-PCM energy:** -115.1586 Hartree
- **Solvation free energy:** -0.00718 Hartree (~ -4.51 kcal/mol)

**Solver Performance**
- **SCI solver total duration:** 311.27 seconds (~5.2 minutes)
- **Number of S-CORE iterations:** 3

**Resource Usage**
| Stage | CPU Time (s) | QPU Time (s) |
|-------|--------------|--------------|
| Mapping | 6.11 | — |
| Optimizing for Hardware | 1.75 | — |
| Waiting for QPU | 0.04 | — |
| Executing QPU | — | 145.41 |
| Post-processing | 312.53 | — |

These results demonstrate that the SQD IEF-PCM workflow successfully recovered the ground-state energy and solvation free energy of methanol using an implicit solvent model on the `ibm_torino` backend.


#### References

\[1] Danil Kaliakin, Akhil Shajan, Fangchun Liang, and Kenneth M. Merz Jr. [Implicit Solvent Sample-Based Quantum Diagonalization](https://pubs.acs.org/doi/10.1021/acs.jpcb.5c01030), The Journal of Physical Chemistry B, 2025, DOI: 10.1021/acs.jpcb.5c01030

\[2] Javier Robledo-Moreno, et al., [Chemistry Beyond Exact Solutions on a Quantum-Centric Supercomputer](https://arxiv.org/abs/2405.05068), arXiv:2405.05068 \[quant-ph].

\[3] Jeffery Yu, et al., [Quantum-Centric Algorithm for Sample-Based Krylov Diagonalization](https://arxiv.org/abs/2501.09702), arXiv:2501.09702 \[quant-ph].

\[4] Keita Kanno, et al., [Quantum-Selected Configuration Interaction: classical diagonalization of Hamiltonians in subspaces selected by quantum computers](https://arxiv.org/abs/2302.11320), arXiv:2302.11320 \[quant-ph].

\[5] Kenji Sugisaki, et al., [Hamiltonian simulation-based quantum-selected configuration interaction for large-scale electronic structure calculations with a quantum computer](https://arxiv.org/abs/2412.07218), arXiv:2412.07218 \[quant-ph].

\[6] Mathias Mikkelsen, Yuya O. Nakagawa, [Quantum-selected configuration interaction with time-evolved state](https://arxiv.org/abs/2412.13839), arXiv:2412.13839 \[quant-ph].

\[7] Herbert, John M. [Dielectric continuum methods for quantum chemistry. WIREs Computational Molecular Science](https://wires.onlinelibrary.wiley.com/doi/10.1002/wcms.1519), 2021, 11, 1759-0876.

\[8] Saki, A. A.; Barison, S.; Fuller, B.; Garrison, J. R.; Glick, J. R.; Johnson, C.; Mezzacapo, A.; Robledo-Moreno, J.; Rossmannek, M.; Schweigert, P. et al. Qiskit addon: sample-based quantum diagonalization, 2024; [https://github.com/Qiskit/qiskit-addon-sqd](https://github.com/Qiskit/qiskit-addon-sqd)

\[9] Asun, Q.; Zhang, X.; Banerjee, S.; Bao, P.; Barbry, M.; Blunt, N. S.; Bogdanov, N. A.; Booth, G. H.; Chen, J.; Cui, Z.-H. PySCF: Python-based Simulations of Chemistry Framework, 2025; [https://github.com/pyscf/pyscf](https://github.com/pyscf/pyscf)

\[10] Kevin J. Sung; et al., FFSIM: Faster simulations of fermionic quantum circuits, 2024. [https://github.com/qiskit-community/ffsim](https://github.com/qiskit-community/ffsim)

